# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 1 2022 10:14AM,243135,10,0085917141,ISDIN Corporation,Released
1,Aug 1 2022 10:14AM,243135,10,0085917145,ISDIN Corporation,Released
2,Aug 1 2022 10:14AM,243135,10,0085917149,ISDIN Corporation,Released
3,Aug 1 2022 10:14AM,243135,10,0085917152,ISDIN Corporation,Released
4,Aug 1 2022 10:14AM,243135,10,0085917221,ISDIN Corporation,Released
5,Aug 1 2022 10:14AM,243135,10,0085917231,ISDIN Corporation,Released
6,Aug 1 2022 10:14AM,243135,10,0085917230,ISDIN Corporation,Released
7,Aug 1 2022 10:14AM,243135,10,0085917232,ISDIN Corporation,Released
8,Aug 1 2022 10:14AM,243135,10,0085917234,ISDIN Corporation,Released
9,Aug 1 2022 10:14AM,243135,10,0085917235,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,243129,Released,1
16,243130,Released,1
17,243131,Released,37
18,243135,Released,46
19,243136,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243129,NaN,1.0
243130,NaN,1.0
243131,NaN,37.0
243135,NaN,46.0
243136,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243108,23.0,9.0
243110,0.0,3.0
243111,1.0,2.0
243114,0.0,1.0
243115,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243108,23,9
243110,0,3
243111,1,2
243114,0,1
243115,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243108,23,9
1,243110,0,3
2,243111,1,2
3,243114,0,1
4,243115,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243108,23,9
1,243110,,3
2,243111,1,2
3,243114,,1
4,243115,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation
46,Aug 1 2022 10:12AM,243136,12,"Uniderm USA, Inc"
47,Aug 1 2022 9:55AM,243111,20,"Exact-Rx, Inc."
50,Aug 1 2022 9:47AM,243131,10,ISDIN Corporation
87,Aug 1 2022 9:41AM,243130,10,"CLINUVEL, Inc."
88,Aug 1 2022 9:32AM,243129,19,"AdvaGen Pharma, Ltd"
89,Aug 1 2022 9:28AM,243127,19,"AdvaGen Pharma, Ltd"
90,Aug 1 2022 9:26AM,243125,10,ISDIN Corporation
111,Aug 1 2022 9:13AM,243123,16,Sartorius Bioprocess Solutions
113,Aug 1 2022 9:11AM,243122,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation,,46
1,Aug 1 2022 10:12AM,243136,12,"Uniderm USA, Inc",,1
2,Aug 1 2022 9:55AM,243111,20,"Exact-Rx, Inc.",1,2
3,Aug 1 2022 9:47AM,243131,10,ISDIN Corporation,,37
4,Aug 1 2022 9:41AM,243130,10,"CLINUVEL, Inc.",,1
5,Aug 1 2022 9:32AM,243129,19,"AdvaGen Pharma, Ltd",,1
6,Aug 1 2022 9:28AM,243127,19,"AdvaGen Pharma, Ltd",1,
7,Aug 1 2022 9:26AM,243125,10,ISDIN Corporation,,21
8,Aug 1 2022 9:13AM,243123,16,Sartorius Bioprocess Solutions,,2
9,Aug 1 2022 9:11AM,243122,12,Hush Hush,44,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation,46,
1,Aug 1 2022 10:12AM,243136,12,"Uniderm USA, Inc",1,
2,Aug 1 2022 9:55AM,243111,20,"Exact-Rx, Inc.",2,1
3,Aug 1 2022 9:47AM,243131,10,ISDIN Corporation,37,
4,Aug 1 2022 9:41AM,243130,10,"CLINUVEL, Inc.",1,
5,Aug 1 2022 9:32AM,243129,19,"AdvaGen Pharma, Ltd",1,
6,Aug 1 2022 9:28AM,243127,19,"AdvaGen Pharma, Ltd",,1
7,Aug 1 2022 9:26AM,243125,10,ISDIN Corporation,21,
8,Aug 1 2022 9:13AM,243123,16,Sartorius Bioprocess Solutions,2,
9,Aug 1 2022 9:11AM,243122,12,Hush Hush,1,44


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation,46,
1,Aug 1 2022 10:12AM,243136,12,"Uniderm USA, Inc",1,
2,Aug 1 2022 9:55AM,243111,20,"Exact-Rx, Inc.",2,1
3,Aug 1 2022 9:47AM,243131,10,ISDIN Corporation,37,
4,Aug 1 2022 9:41AM,243130,10,"CLINUVEL, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation,46.0,NaN
1,Aug 1 2022 10:12AM,243136,12,"Uniderm USA, Inc",1.0,NaN
2,Aug 1 2022 9:55AM,243111,20,"Exact-Rx, Inc.",2.0,1.0
3,Aug 1 2022 9:47AM,243131,10,ISDIN Corporation,37.0,NaN
4,Aug 1 2022 9:41AM,243130,10,"CLINUVEL, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation,46.0,0.0
1,Aug 1 2022 10:12AM,243136,12,"Uniderm USA, Inc",1.0,0.0
2,Aug 1 2022 9:55AM,243111,20,"Exact-Rx, Inc.",2.0,1.0
3,Aug 1 2022 9:47AM,243131,10,ISDIN Corporation,37.0,0.0
4,Aug 1 2022 9:41AM,243130,10,"CLINUVEL, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2431215,121.0,0.0
12,486258,2.0,44.0
16,243123,2.0,0.0
19,729364,10.0,24.0
20,243111,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2431215,121.0,0.0
1,12,486258,2.0,44.0
2,16,243123,2.0,0.0
3,19,729364,10.0,24.0
4,20,243111,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,121.0,0.0
1,12,2.0,44.0
2,16,2.0,0.0
3,19,10.0,24.0
4,20,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,121.0
1,12,Released,2.0
2,16,Released,2.0
3,19,Released,10.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,0.0,44.0,0.0,24.0,1.0
Released,121.0,2.0,2.0,10.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,0.0,44.0,0.0,24.0,1.0
1,Released,121.0,2.0,2.0,10.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,0.0,44.0,0.0,24.0,1.0
1,Released,121.0,2.0,2.0,10.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()